In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool
from functools import partial
import Feature_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\Stock\\TWSE_Stock_fundamental.csv', converters={'ts': str, 'StockName': str, 'StockNo': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,StockName,StockNo,close,high,low,open,return,total,ts,vol,year,total_num,eliminate_period,foreign_buy,investment_buy,dealer_buy,fixed_close
0,台泥,1101,21.00,21.00,20.40,20.70,0.10,417236030.0,2004-12-20,20105028.0,2004,NaN,0,8827.0,0.0,101.0,7.80
1,台泥,1101,20.20,20.20,19.70,19.70,0.50,100967115.0,2005-08-12,5037589.0,2005,NaN,0,2013.0,3.0,545.0,7.50
2,台泥,1101,19.60,20.40,19.50,20.40,-0.80,420293307.0,2005-08-10,21295902.0,2005,NaN,0,-10194.0,-1084.0,-2169.0,7.28
3,台泥,1101,20.40,20.40,20.20,20.30,0.10,121245861.0,2005-08-24,5970108.0,2005,NaN,0,1696.0,1002.0,-16.0,7.58
4,台泥,1101,20.40,20.55,20.20,20.50,-0.20,147516136.0,2005-08-09,7238224.0,2005,NaN,0,-2010.0,-2.0,-375.0,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874551,世紀鋼,9958,11.30,11.80,11.10,11.80,-0.50,1950500.0,2008-07-08,170000.0,2008,NaN,0,-23.0,0.0,-4.0,7.88
2874552,世紀鋼,9958,19.70,19.80,19.40,19.50,0.30,14390595.0,2008-04-15,732635.0,2008,NaN,0,0.0,0.0,0.0,13.73
2874553,世紀鋼,9958,19.55,20.00,19.55,20.00,-0.15,18114525.0,2008-04-16,919500.0,2008,NaN,0,0.0,0.0,0.0,13.63
2874554,世紀鋼,9958,9.96,10.00,9.60,9.97,-0.01,2673890.0,2008-09-25,270000.0,2008,NaN,0,44.0,0.0,1.0,7.29


In [3]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]

output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.get_technical_indicators, df_list), total=len(df_list)), 1):
            output_list.append(x)
                
df = pd.concat(output_list, axis=0)
df

,StockName,StockNo,close,high,low,open,return,total,ts,vol,...,investment_buy_kurtosis,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO%k5,SO%d3
0,嘉泥,1103,21.847801,22.465270,21.487662,21.978549,NaN,333925503.0,2004-02-11,1.695612e+07,...,NaN,NaN,NaN,NaN,NaN,0.977608,0.977608,0.000000,0.000000,0.000000
1,嘉泥,1103,21.897940,22.506702,21.541265,22.025166,-0.90,246048766.0,2004-02-12,1.423377e+07,...,NaN,NaN,NaN,32792.061209,NaN,8.396205,4.934193,0.000000,0.000000,0.000000
2,嘉泥,1103,21.948814,22.548722,21.595610,22.072402,0.40,333309529.0,2004-02-13,1.558382e+07,...,NaN,NaN,NaN,36360.930024,69152.991233,8.405717,6.248853,0.000000,0.000000,0.000000
3,嘉泥,1103,22.000254,22.591188,21.650509,22.120088,1.00,515700813.0,2004-02-16,1.721540e+07,...,NaN,NaN,NaN,40539.782945,76900.712969,8.415183,6.903225,0.000000,0.000000,0.000000
4,嘉泥,1103,22.054429,22.635928,21.708364,22.170379,-0.10,333141524.0,2004-02-17,1.614430e+07,...,NaN,NaN,NaN,39944.302832,80484.085777,8.426541,7.294147,0.000000,-6.307874,-3.255677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,潤泰新,9945,42.597097,42.882676,42.478737,42.769965,0.00,42735700.0,2019-09-17,9.913275e+05,...,14.891503,0.774622,-0.069637,-2939.578552,-5810.041361,0.403939,0.448760,0.394791,0.173960,0.230880
3855,潤泰新,9945,42.508437,42.815835,42.407700,42.711107,-0.30,68377912.0,2019-09-18,1.560745e+06,...,14.379924,0.959324,0.222628,-3248.485107,-6188.063659,0.408136,0.443682,0.347899,0.146203,0.188541
3856,潤泰新,9945,42.456466,42.772982,42.365112,42.673485,-0.55,54539822.0,2019-09-19,1.289888e+06,...,14.379924,0.959324,0.222628,-1577.016568,-4825.501675,0.407870,0.439206,0.321718,0.133362,0.160952
3857,潤泰新,9945,42.423662,42.743618,42.336151,42.646875,-0.50,116365981.0,2019-09-20,2.742475e+06,...,7.977569,0.946702,0.183959,-2119.021081,-3696.037649,0.407467,0.435238,0.303942,0.137104,0.149028


In [4]:
df.columns

Index(['StockName', 'StockNo', 'close', 'high', 'low', 'open', 'return',
       'total', 'ts', 'vol', 'year', 'total_num', 'foreign_buy',
       'investment_buy', 'dealer_buy', 'fixed_close', 'VWAP', 'origin_close',
       'eliminate', 'ma7', 'ma21', '26ema', '12ema', 'MACD', 'upper_band',
       'lower_band', 'fixed_close_skew', 'fixed_close_kurtosis',
       'foreign_buy_skew', 'foreign_buy_kurtosis', 'investment_buy_skew',
       'investment_buy_kurtosis', 'dealer_buy_skew', 'dealer_buy_kurtosis',
       'pvt_current', 'pvt', 'TR', 'ATR', 'RSI_15', 'SO%k5', 'SO%d3'],
      dtype='object')

In [5]:
df.to_csv('D:\\庫存健診開發\\data\\Stock\\TWSE_Stock_feature.csv', index=False)

In [1]:
sum([1,2,3,4])

10